# Processing Word with XML

## Replace centent

In [13]:
import zipfile
templateDocx = zipfile.ZipFile("data/Template.docx")
newDocx = zipfile.ZipFile("data/NewDocument.docx", "a")


for file in templateDocx.filelist:
    templateDocx.extract(file.filename, "data")

with open("data/word/document.xml", encoding='utf-8') as docXmlFile:
    docXmlStr =  docXmlFile.read()
    # print(docXmlStr)
       
replaceText={'第五节':'这是替换的文本'} 
for key in replaceText.keys():
    new_docXmlStr = docXmlStr.replace(str(key), str(replaceText.get(key)))

with open("data/word/new_document.xml", "w+", encoding='utf-8') as new_docXmlFile:
    new_docXmlFile.write(new_docXmlStr)

for file in templateDocx.filelist:
    if not file.filename == "word/document.xml":
        newDocx.writestr(file.filename, templateDocx.read(file))

newDocx.write("data/word/new_document.xml", "word/document.xml")

templateDocx.close()
newDocx.close()

## Processing fontnote with footnotes.xml

In [14]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open("data/word/footnotes.xml", encoding='utf-8'), 'xml')
# print(soup.prettify())
for footnote in soup.find_all('w:footnote'):
    footnote_id = footnote.get('w:id')
    print( footnote_id,)
    print(footnote.text, '\n ','--'*10)


-1
 
  --------------------
0
 
  --------------------
1
 .《关于成立‘江苏东大特种基础工程开发公司’的报告》，1-2-1。 
  --------------------
2
 .《联营协议书》，1-2-1。 
  --------------------
3
 .《关于成立“江苏东大特种基础工程开发公司”的决定》，1-2-1。 
  --------------------
4
 .验字（94）6号验资证明，1-3-2。 
  --------------------
5
 .《国有资产产权登记表附表》，1-2-1。 
  --------------------
6
 .《审计报告》（XYZH/2015NJA30090号），1-4-4-1。 
  --------------------
7
 《评估报告》（天源评报字[2015]第0199号），1-4-4-2。 
  --------------------
8
 《验资报告》（XYZH[20l5]NJA30092号），1-4-4-3。 
  --------------------


In [15]:
for footnote in soup.find_all('w:footnote'):
    for w_t in footnote.find_all('w:t'):
        if  "关于成立" in w_t:
            print(w_t)
            w_t.insert(1,'测试')
            print(w_t)

with open('data/word/new_footnotes.xml', 'w',  encoding='utf-8') as fp:
    fp.write(soup.prettify())

<w:t>关于成立</w:t>
<w:t>关于成立测试</w:t>


# Processing Word with python-docx

## get paragraph text

In [7]:
from docx import Document
import re
import multiprocessing as mp


class Docx_extract:
    def __init__(self, fp):
        self.document = Document(fp)

    def get_paragraph(self):
        '''
        Get the word document paragraph number and paragraph text that doesn't include heading.

        Parameters
        ----------
        f_p: docx file path

        Returns
        ----------
        out: generator
        Each element is A tuple containing (paragraph_number, paragraph.text)
        '''
        for  paragraph_number, paragraph in enumerate(self.document.paragraphs):
            if (paragraph.style.name =='Normal' or re.findall('正文', paragraph.style.name)) and len(paragraph.text.strip()) !=0:
                yield  paragraph_number, paragraph.text.strip()
    




if __name__=="__main__":
   
    fp = 'data/Template.docx'
    docx_extract = Docx_extract(fp)
    paragraph_texts = docx_extract.get_paragraph()

In [214]:
list(res)

[]

# Docx to byte stream and byte stream to docx 

In [5]:
import base64

fp = 'data/Template.docx'
with open(fp,'rb') as f:
    res = base64.b64encode(f.read())

docx = base64.b64decode(res)

with open('test.docx','wb') as f:
    f.write(docx)